# Option Pricing with with PyTorch 

Copyright Matthias Groncki, 2018

This is a port of one of my previous blog posts about using TensorFlow to price options.

After using PyTorch for another project, i was impressed how straight forward it is, so I've decided to revisit my previous examples and use PyTorch this time

In [1]:
import numpy as np
import torch
import tensorflow as tf
import scipy.stats as stats

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Plain Vanillas

Lets start with plain vanillas in a Black Scholes World. 

### Numpy Implementation

I am using the same implementation as in the TensorFlow notebook:

In [2]:
## Plain Vanilla Call in TensorFlow

def blackScholes_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = stats.norm.cdf
    d_1 = (np.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))
    d_2 = d_1 - sigma*np.sqrt(dt)
    return S*Phi(d_1) - K*np.exp(-r*dt)*Phi(d_2)

In [3]:
%%timeit
S_0 = 100
K = 101
T = 1
sigma = 0.3
r = 0.01

npv_numpy = blackScholes_py(S_0, K, T, sigma, r)

281 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


And as expected its super fast. No suprises here

### PyTorch Implementation

There are only minimal code changes compared to the numpy version required. In the actual pricing function we just need to replace ```np``` with ```torch``` and exchange the cdf function to use the PyTorch one and we have to convert our input into ```torch.tensor```.

In [4]:
def blackScholes_pyTorch(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = torch.distributions.Normal(0,1).cdf
    d_1 = (torch.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*torch.sqrt(dt))
    d_2 = d_1 - sigma*torch.sqrt(dt)
    return S*Phi(d_1) - K*torch.exp(-r*dt)*Phi(d_2)

In [5]:
%%timeit
S_0 = torch.tensor([100.],requires_grad=True)
K = torch.tensor([101.],requires_grad=True)
T = torch.tensor([1.],requires_grad=True)
sigma = torch.tensor([0.3],requires_grad=True)
r = torch.tensor([0.01],requires_grad=True)
npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)

283 µs ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Seems the PyTorch version is even faster as the pure numpy version

#### Greeks in PyTorch

We just need to call the ```.backward()``` operator of the tensor which stores the prices and we can access the greeks with the ```.grad``` properity.

In [6]:
%%timeit
S_0 = torch.tensor([100.],requires_grad=True)
K = torch.tensor([101.],requires_grad=True)
T = torch.tensor([1.],requires_grad=True)
sigma = torch.tensor([0.3],requires_grad=True)
r = torch.tensor([0.01],requires_grad=True)
npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)
npv_pytorch.backward()
delta_pytorch = S_0.grad
vega_pytorch = sigma.grad
rho_pytorch = r.grad
theta_pytorch = T.grad
digital_pytoch = -K.grad

639 µs ± 9.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Its almost 2.5 times slower but gives us five greeks. A naive finite-difference approximation would costs us at least 5 calculations and would be only an numerical approximation. Here we have 'exact' derivates.

### TensorFlow implementation

Using the same code as in the original notebook (but I removed the calculation of the 2nd order greeks. There is a bit of overhead for constructing the computational graph.

In [7]:
def blackScholes_tf_pricer():
    # Build the static computational graph
    S = tf.placeholder(tf.float32)
    K = tf.placeholder(tf.float32)
    dt = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    Phi = tf.distributions.Normal(0.,1.).cdf
    d_1 = (tf.log(S / K) + (r+sigma**2/2)*dt) / (sigma*tf.sqrt(dt))
    d_2 = d_1 - sigma*tf.sqrt(dt)
    npv =  S*Phi(d_1) - K*tf.exp(-r*dt)*Phi(d_2)
    greeks = tf.gradients(npv, [S, sigma, r, K, dt])
    def execute_graph(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
        with tf.Session() as sess:
            res = sess.run([npv, greeks], 
                           {
                               S: S_0,
                               K : strike,
                               r : riskfree_rate,
                               sigma: implied_vol,
                               dt: time_to_expiry})
        return res
    return execute_graph 

In [8]:
%%timeit
S_0 = 100
K = 101
T = 1
sigma = 0.3
r = 0.01
tf_pricer = blackScholes_tf_pricer()
npv_numpy_tf = tf_pricer(S_0, K, T, sigma, r)

820 ms ± 99.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Its much slower than the numpy and the PyTorch implementaion. Maybe my implementation is just bad.

#### Second order greeks in Pytorch

We using the same example as before but now we want to calculate the 2nd order greeks. Thats need
to create a computational graph of the gradient. We use the function ```.grad()``` from the autograd module.

In [9]:
S_0 = torch.tensor([100.],requires_grad=True)
K = torch.tensor([101.],requires_grad=True)
T = torch.tensor([1.],requires_grad=True)
sigma = torch.tensor([0.3],requires_grad=True)
r = torch.tensor([0.01],requires_grad=True)
npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)

#### Gamma

In [10]:
gradient = torch.autograd.grad(npv_pytorch, S_0, create_graph=True)

In [11]:
delta, =  gradient
delta

tensor([ 0.5597])

In [18]:
delta.backward(retain_graph=True)
print('Delta: ', delta)
print('Gamma', S_0.grad)

Delta:  tensor([ 0.5597])
Gamma tensor(1.00000e-02 *
       [ 2.6298])


## Monte Carlo Pricing for Single Barrier Option

### Numpy Implementation

In [13]:
def monte_carlo_down_out_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, steps, samples):
    stdnorm_random_variates = np.random.randn(samples, steps)
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    B_shift = B*np.exp(0.5826*sigma*np.sqrt(dt))
    S_T = S * np.cumprod(np.exp((r-sigma**2/2)*dt+sigma*np.sqrt(dt)*stdnorm_random_variates), axis=1)
    non_touch = (np.min(S_T, axis=1) > B_shift)*1
    call_payout = np.maximum(S_T[:,-1] - K, 0)
    npv = np.mean(non_touch * call_payout)
    return np.exp(-time_to_expiry*r)*npv

In [21]:
%%timeit
monte_carlo_down_out_py(100., 110., 2., 0.2, 0.03, 90., 1000, 100000)

4.78 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### PyTorch Implementation

In [15]:
def monte_carlo_down_out_torch(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, steps, samples):
    stdnorm_random_variates = torch.distributions.Normal(0,1).sample((samples, steps))
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    B_shift = B*torch.exp(0.5826*sigma*torch.sqrt(dt))
    S_T = S * torch.cumprod(torch.exp((r-sigma**2/2)*dt+sigma*torch.sqrt(dt)*stdnorm_random_variates), dim=1)
    non_touch = torch.min(S_T, dim=1)[0] > B_shift
    call_payout = S_T[:,-1] - K
    call_payout[call_payout<0]=0
    npv = torch.mean(non_touch.type(torch.FloatTensor) * call_payout)
    return torch.exp(-time_to_expiry*r)*npv

In [20]:
%%timeit
S = torch.tensor([100.],requires_grad=True)
K = torch.tensor([110.],requires_grad=True)
T = torch.tensor([2.],requires_grad=True)
sigma = torch.tensor([0.2],requires_grad=True)
r = torch.tensor([0.03],requires_grad=True)
B = torch.tensor([90.],requires_grad=True)
monte_carlo_down_out_torch(S, K, T, sigma, r, B, 1000, 100000)

3.09 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
S = torch.tensor([100.],requires_grad=True)
K = torch.tensor([110.],requires_grad=True)
T = torch.tensor([2.],requires_grad=True)
sigma = torch.tensor([0.2],requires_grad=True)
r = torch.tensor([0.03],requires_grad=True)
B = torch.tensor([90.],requires_grad=True)
npv_torch_mc = monte_carlo_down_out_torch(S, K, T, sigma, r, B, 1000, 100000)
npv_torch_mc.backward()

7.34 s ± 229 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
